In [1]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import os, sys, h5py
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import rcParams
% matplotlib inline

sys.path.append('/Users/juliankimura/Desktop/deepomics')
import deepomics.neuralnetwork as nn
from deepomics import learn, utils, metrics, visualize, explore
from deepomics.build_network import build_network 

np.random.seed(247) # for reproducibility

In [2]:
def load_data(data_path):
    dataset = h5py.File(data_path,'r')

    print("loading training data")
    X_train = np.expand_dims(dataset['X_train'], axis=3)
    y_train = np.array(dataset['Y_train'])

    print("loading validation data")  
    X_valid = np.expand_dims(dataset['X_valid'], axis=3)
    y_valid = np.array(dataset['Y_valid'])

    print("loading test data")
    X_test = np.expand_dims(dataset['X_test'], axis=3)
    y_test = np.array(dataset['Y_test'])

    train = (X_train, y_train)
    valid = (X_valid, y_valid)
    test = (X_test, y_test)

    return train, valid, test

data_path = '.'
file_name = 'K562_CEBPB_200_dinuc.h5'
train, valid, test = load_data(os.path.join(data_path, file_name))



loading training data
loading validation data
loading test data


In [3]:
def model(input_shape, output_shape):

    # create model
    layer1 = {'layer': 'input',
                'shape': input_shape,
                'name': 'input'
                }
    layer2 = {'layer': 'conv1d', 
                'num_filters': {'start': 20, 'bounds': [1, 200], 'scale': 20},
                'filter_size': {'start': 19, 'bounds': [5, 27], 'scale': 10, 'multiples': 2, 'offset': 1},
                'norm': 'batch',
                'padding': 'same',
                'activation': 'relu',
                'pool_size': {'start': 20, 'bounds': [1, 200], 'scale': 10, 'multiples': 4},
                }
    layer3 = {'layer': 'dense', 
                'num_units': {'start': 64, 'bounds': [16, 1000], 'scale': 50},
                'norm': 'batch',
                'activation': 'relu',
                }
    layer3 = {'layer': 'dense', 
                'num_units': output_shape[1],
                'activation': 'sigmoid',
                }

    #from tfomics import build_network
    model_layers = [layer1, layer2, layer3]

    # optimization parameters
    optimization = {"objective": "binary",
                    "optimizer": "adam",
                    "learning_rate": 0.001,
                    "l2": 1e-6
                    #"learning_rate": {'start': -3, 'bounds': [-4, -1], 'scale': 1.5, 'transform': 'log'},      
                    #"l2": {'start': -6, 'bounds': [-8, -2], 'scale': 3, 'transform': 'log'},
                    # "l1": 0, 
                    }
    return model_layers, optimization

# build network
input_shape = list(train[0].shape)
input_shape[0] = None
output_shape = train[1].shape
model_layers, optimization = model(input_shape, output_shape)


In [4]:
output_name = 'test'
utils.make_directory(data_path, 'Results')
file_path = os.path.join(data_path, 'Results', output_name)
nnoptimizer = explore.NeuralOptimizer(model_layers, output_shape, optimization)

In [5]:
nnoptimizer.optimize(data={'train': train, 'valid': valid}, num_trials=10, batch_size=128,
                    num_epochs=10, verbose=0)

---------------------------------------------------------
Running baseline model

Model layers:
name: input
num_filters: 20
filter_size: 19
pool_size: 20
num_units: 1

Optimization:
learning_rate: 0.001
l2: 1e-06

compiling model


TypeError: test_model() got an unexpected keyword argument 'verbose'